## Data Processing

In [1]:
import pandas as pd
import numpy as np
import pandas_datareader.data as web
import sqlite3 as db
import requests
import requests_cache
requests_cache.install_cache('cache')

### 1. Scraping ETF data from the web and build SQLite3 database for tikcers

In [ ]:
# list of webpages
htmls = '''
http://www.etf.com/channels/bond-etfs
http://www.etf.com/channels/mlp-etfs
http://www.etf.com/channels/silver-etfs
http://www.etf.com/channels/china-etfs
http://www.etf.com/channels/muni-etfs
http://www.etf.com/channels/us-broad-market-bond-etfs
http://www.etf.com/channels/dividend-etfs
http://www.etf.com/channels/natural-gas-etfs
http://www.etf.com/channels/global-bond-etfs
http://www.etf.com/channels/oil-etfs
http://www.etf.com/channels/treasury-etfs
http://www.etf.com/channels/gold-etfs
http://www.etf.com/channels/reit-etfs
http://www.etf.com/channels/high-dividend-yield-etfs
http://www.etf.com/channels/japan-etfs
http://www.etf.com/channels/smart-beta-etfs
http://www.etf.com/etf-lists/alternatives-etfs
http://www.etf.com/etf-lists/asset-allocation-etfs
http://www.etf.com/etf-lists/currency-etfs
http://www.etf.com/etf-lists/fixed-income-etfs
http://www.etf.com/channels/alpha-seeking-etfs
http://www.etf.com/channels/basic-materials-etfs
http://www.etf.com/channels/consumer-cyclicals-etfs
http://www.etf.com/channels/consumer-non-cyclicals-etfs
http://www.etf.com/channels/energy-etfs
http://www.etf.com/channels/extended-market-etfs
http://www.etf.com/channels/financials-etfs
http://www.etf.com/channels/health-care-etfs
http://www.etf.com/channels/high-dividend-yield-etfs
http://www.etf.com/channels/industrials-etfs
http://www.etf.com/channels/real-estate-etfs
http://www.etf.com/channels/small-cap-etfs
http://www.etf.com/channels/technology-etfs
http://www.etf.com/channels/telecommunications-etfs
http://www.etf.com/channels/theme-etfs
http://www.etf.com/channels/total-market-etfs
http://www.etf.com/channels/utilities-etfs
http://www.etf.com/channels/asia-pacific-etfs
http://www.etf.com/channels/developed-markets-etfs
http://www.etf.com/channels/emerging-markets-etfs
http://www.etf.com/channels/europe-etfs
http://www.etf.com/channels/global-etfs
http://www.etf.com/channels/global-ex-us-etfs
http://www.etf.com/channels/latin-america-etfs
http://www.etf.com/channels/middle-east-and-africa-etfs
'''

data_raw = []

for i in htmls.split():
    print("Scraping data from {}.".format(i))
    df = pd.read_html(requests.get(i,headers={'user-agent': 'Mozilla/5.0'}).text)
    #df holds multiple DataFrames - index [11] or index[5] is the DataFrame we are interested in 
    #the scraped data has slightly different headings across different webpages 
    #so we reset the column headings to be identical for each DataFrame  
    #these names also match the columns we will set up in out SQLite database
    try:
        df_want = df[11]
        df_want.columns = ['Fund Name','Ticker','Asset Class', 'Strategy','Region','Geography','Category','Focus','Niche', 'Inverse','Leveraged','ETN','Underlying Index','Index Provider','Selection Criteria','Weighting Scheme','Active per SEC']
    except IndexError:
        df_want = df[5]
        df_want.columns = ['Fund Name','Ticker','Asset Class', 'Strategy','Region','Geography','Category','Focus','Niche', 'Inverse','Leveraged','ETN','Underlying Index','Index Provider','Selection Criteria','Weighting Scheme','Active per SEC']

    data_raw.append(df_want)
    
tickers_df = pd.concat(data_raw).reset_index(drop = True).drop_duplicates()
tickers_df.head()

In [ ]:
cnx = db.connect('/Users/Rachel/Dropbox/STA237 Project/tickers_database.db')
cur = cnx.cursor()

#remove table if already exists and any data it contains
cur.execute('DROP TABLE IF EXISTS ETFtable;')
 
#create the table within the database
sql = '''CREATE TABLE ETFtable ('Fund Name' TEXT, 'Ticker' TEXT, 'Asset Class' TEXT, 'Strategy' TEXT,
                                'Region' TEXT, 'Geography' TEXT, 'Category' TEXT, 'Focus' TEXT,
                                'Niche' TEXT, 'Inverse' TEXT, 'Leveraged' TEXT, 'ETN' TEXT, 
                                'Underlying Index' TEXT, 'Index Provider' TEXT, 'Selection Criteria' TEXT, 
                                'Weighting Scheme' TEXT, 'Active per SEC' TEXT)''' 
cur.execute(sql)

#upload table
tickers_df.to_sql(name='ETFtable', con=cnx, if_exists = 'append', index=False)

cnx.close()

### 2. Get price data of assets from Yahoo Finance and build SQLite3 database

In [ ]:
cnx = db.connect('/Users/Rachel/Dropbox/STA237 Project/tickers_database.db')
cur = cnx.cursor()

sql1 = 'SELECT Ticker FROM ETFtable WHERE Focus = "Crude Oil" AND "Asset Class" = "Commodities"'
cmd = pd.read_sql(sql1, con=cnx)['Ticker'].values.tolist()

sql2 = 'SELECT Ticker FROM ETFtable WHERE Focus = "Energy" AND "Asset Class" = "Equity"'
eqt = pd.read_sql(sql2, con=cnx)['Ticker'].values.tolist()

cnx.close()

In [ ]:
def get_price_data(ticker, start, end):
    df = pd.DataFrame(web.DataReader(ticker, 'yahoo', start, end)['Adj Close'])
    df['Ticker'] = [ticker] * len(df)
    return df

In [ ]:
start_date, end_date = '2014/06/25', '2017/11/25'
data_list = []
res_list = []
for e in eqt + cmd:
    try:
        df = get_price_data(e, start_date, end_date).reset_index()
        data_list.append(df)
        print(e)
    except:
        res_list.append(e)

In [ ]:
df1 = pd.concat(data_list)
df1['Date'] = [str(dt)[:10] for dt in df1['Date'].values]

In [ ]:
import glob
import re

file_names = glob.glob('./Data/*.csv')
data_list2 = []
for f in file_names:
    ticker = f.split('/')[2].split('.')[0]
    print(ticker)
    if ticker not in df1.columns.values:
        df = pd.DataFrame(pd.read_csv(f).set_index('Date')['Adj Close']).reset_index()
        df['Ticker'] = [ticker] * len(df)
        data_list2.append(df)

In [ ]:
df2 = pd.concat(data_list2)
df = pd.concat([df1, df2])
df.head()

In [ ]:
# Build price data base
cnx = db.connect('/Users/Rachel/Dropbox/STA237 Project/tickers_database.db')
cur = cnx.cursor()

#remove table if already exists and any data it contains
cur.execute('DROP TABLE IF EXISTS Pricetable;')
 
#create the table within the database
sql = '''CREATE TABLE Pricetable ('Date' TEXT, 'Adj Close' FLOAT, 'Ticker' TEXT)''' 
cur.execute(sql)

#upload table
df.to_sql(name='Pricetable', con=cnx, if_exists = 'append', index=False)

cnx.close()